In [1]:
import numpy as np
import main

In [2]:
import time

In [3]:
M_ls = []

In [4]:
rod_depth_1 = 160
rod_depth_2 = 160
total_height = 160
# Main material
M = main.material_FDM(mat_id=0)
M.set_XS('XS_scatter',[[0,0.017555500],[0,0]])
M.set_XS('Diffusion_coef',[1.423913000,0.356306000])
M.set_XS('XS_absorption',[0.010402060,0.087662170])
M.set_XS('XS_nu_fission',[0.006477691,0.112732800])
M.set_XS('XS_fission_spectrum',[1.0,0.0])
M_ls.append(M)

# Control rod 1
M = main.material_FDM(mat_id=1)
M.set_XS('XS_scatter',[[0,0.017555500],[0,0]])
M.set_XS('Diffusion_coef',[1.423913000,0.356306000])
M.set_XS('XS_absorption',[(0.010952060*rod_depth_1+0.010402060*(total_height-rod_depth_1))/total_height,(0.091462170*rod_depth_1+0.087662170*(total_height-rod_depth_1))/total_height])
M.set_XS('XS_nu_fission',[0.006477691,0.112732800])
M.set_XS('XS_fission_spectrum',[1.0,0.0])
M_ls.append(M)

M = main.material_FDM(mat_id=2)
M.set_XS('XS_scatter',[[0,0.01717768],[0,0]])
M.set_XS('Diffusion_coef',[1.425611000,0.350574000])
M.set_XS('XS_absorption',[0.010992630,0.099256340])
M.set_XS('XS_nu_fission',[0.007503284,0.137800400])
M.set_XS('XS_fission_spectrum',[1.0,0.0])
M_ls.append(M)

M = main.material_FDM(mat_id=3)
M.set_XS('XS_scatter',[[0,0.02759693],[0,0]])
M.set_XS('Diffusion_coef',[1.634227000,0.264002000])
M.set_XS('XS_absorption',[0.002660573,0.049363510])
M.set_XS('XS_nu_fission',[0.0,0.0])
M.set_XS('XS_fission_spectrum',[1.0,0.0])
M_ls.append(M)

# Control rod 2
M = main.material_FDM(mat_id=4)
M.set_XS('XS_scatter',[[0,0.017555500],[0,0]])
M.set_XS('Diffusion_coef',[1.423913000,0.356306000])
# M.set_XS('XS_absorption',[0.010952060,0.091462170])
M.set_XS('XS_absorption',[(0.010952060*rod_depth_2+0.010402060*(total_height-rod_depth_2))/total_height,(0.091462170*rod_depth_2+0.087662170*(total_height-rod_depth_2))/total_height])
M.set_XS('XS_nu_fission',[0.006477691,0.112732800])
M.set_XS('XS_fission_spectrum',[1.0,0.0])
M_ls.append(M)

In [5]:
precursor_1 = main.precursor_FDM(delay_group_num=6)
precursor_1.set_beta([0.0002470,0.0013845,0.0012220,0.0026455,0.0008320,0.0001690])
precursor_1.set_lambda([0.0127,0.0317,0.1150,0.3110,1.4000,3.8700])

In [6]:
geo = main.geometry_FDM(x_block=6,y_block=6)
layout = np.array([[5,1,1,2,3,4],[1,1,1,1,3,4],[1,1,5,1,3,4],[2,1,1,3,3,4],[3,3,3,3,4,4],[4,4,4,4,4,4]],dtype=np.int32)
layout = layout-1

geo.set_block_mat_by_array(layout)

geo.set_block_size(x_size=[10,20,20,20,20,20],y_size=[10,20,20,20,20,20])
geo.set_discretized_num(x_dim=[10,20,20,20,20,20],y_dim=[10,20,20,20,20,20])
geo.check_blocks()

Material layout:
[[4 0 0 1 2 3]
 [0 0 0 0 2 3]
 [0 0 4 0 2 3]
 [1 0 0 2 2 3]
 [2 2 2 2 3 3]
 [3 3 3 3 3 3]]
Block size:
x:[10, 20, 20, 20, 20, 20]
y:[10, 20, 20, 20, 20, 20]
Discretization number:
x:[10, 20, 20, 20, 20, 20]
y:[10, 20, 20, 20, 20, 20]


In [7]:
case = main.solver_FDM(folder_name='LMW_2D_Case2_adaptive_01_normal',group_num=2)
case.add_material(M_ls)
case.set_geometry(geo)
case.set_geometry(geo,transient_mode=True)
case.set_precursor(precursor_1)
case.set_neutron_velocity([1.25e7,2.5e5])

In [8]:
# def material_update_subroutine(mat_ls,case,time_step):
#     total_height = 160
#     # Control rod 1
#     if time_step <=26.5:
#         rod_depth_1 = 80-time_step*80/26.5
#     else:
#         rod_depth_1 = 0.0
#     mat_ls[1].set_XS('XS_absorption',[(0.010952060*rod_depth_1+0.010402060*(total_height-rod_depth_1))/total_height,(0.091462170*rod_depth_1+0.087662170*(total_height-rod_depth_1))/total_height])
#     case.update_material(1,M_ls[1],mat_type='real')
#     # Control rod 2
#     if time_step <=47.5 and time_step>=7.5:
#         rod_depth_2 = 3*(time_step-7.5)# Insert control rod
#     elif time_step<7.5:
#         rod_depth_2 = 0
#     else:
#         rod_depth_2 = 120
#     mat_ls[4].set_XS('XS_absorption',[(0.010952060*rod_depth_2+0.010402060*(total_height-rod_depth_2))/total_height,(0.091462170*rod_depth_2+0.087662170*(total_height-rod_depth_2))/total_height])
#     case.update_material(4,M_ls[4],mat_type='real')
#     return case,mat_ls
def material_update_subroutine(mat_ls,case,time_step):
    total_height = 160
    # Control rod 1
    if time_step <=0.1:
        rod_depth_1 = 160-time_step*160/0.1
    elif time_step>=0.4:
        rod_depth_1 = 160*(time_step-0.4)/0.6
    else:
        rod_depth_1 = 0
    mat_ls[1].set_XS('XS_absorption',[(0.010952060*rod_depth_1+0.010402060*(total_height-rod_depth_1))/total_height,(0.091462170*rod_depth_1+0.087662170*(total_height-rod_depth_1))/total_height])
    case.update_material(1,M_ls[1],mat_type='real')
    # Control rod 2
    if time_step <=0.1:
        rod_depth_2 = 160-time_step*160/0.1
    elif time_step>0.1 and time_step<=0.2:
        rod_depth_2 = 0
    elif time_step>0.2 and time_step<0.8:
        rod_depth_2 = 160*(time_step-0.2)/0.6
    else:
        rod_depth_2 = 160    
    mat_ls[4].set_XS('XS_absorption',[(0.010952060*rod_depth_2+0.010402060*(total_height-rod_depth_2))/total_height,(0.091462170*rod_depth_2+0.087662170*(total_height-rod_depth_2))/total_height])
    case.update_material(4,M_ls[4],mat_type='real')
    return case,mat_ls

In [9]:
# Key time steps
fix_time_steps = np.array([0.0,0.1,0.2,0.4,0.8,1.0]) 
time_index = 0
time_step = fix_time_steps[0]
# Max and min time steps
max_time_step = 5e-2
min_time_step = 1e-3
# Time pointer
key_time_pointer = 1
#Error tolerance
error_tolerance = 1e-3
while True:
    if time_index==0:
        # To get initial flux distribution
        case.solve_source_iter_correct(max_iter=100,k_tolerance=1e-5,flux_tolerance=1e-4,initial_k=1.0)
        case.solve_source_adjoint_iter_correct(max_iter=100,k_tolerance=1e-5,flux_tolerance=1e-4,initial_k=1.0)
        time_start=time.time()
        case.initial_dynamics(time_steps=fix_time_steps,transient_algorithm='SCM_adaptive',vtk_save=True,max_time_step=max_time_step,min_time_step=min_time_step)
    else:
        # Update material
        case,M_ls=material_update_subroutine(M_ls,case,time_step)
        time_interval = case.evaluate_adaptive_time_step_exact(time_index,pre_time_interval=time_interval,error_tolerance=error_tolerance,mode='Normal-estimate')
        time_step = time_step_last+time_interval
        if time_step>=fix_time_steps[key_time_pointer]:
            key_time_pointer+=1        
        print('Time={} (s)'.format(time_step))
        # Re-update material
        case,M_ls=material_update_subroutine(M_ls,case,time_step)
        case.solve_transient_SCM_PKE_predictor(time_index,time_interval,max_iter=100,k_tolerance=1e-6,flux_tolerance=1e-4,k_outer_tolerance=1e-6)
    # Move time pointer
    time_index += 1
    time_step_last = time_step
    # End or not
    if time_step_last>=fix_time_steps[-1]:
        break
    # Pre-choose the last time step
    if fix_time_steps[key_time_pointer]-time_step_last>=max_time_step:
        time_step += max_time_step 
        time_interval = max_time_step
    else:
        time_step = fix_time_steps[key_time_pointer]
        time_interval = fix_time_steps[key_time_pointer]-time_step_last
        

time_end=time.time()
print('totally cost:',time_end-time_start)

D:\Users\85136\anaconda3\lib\site-packages\scipy\sparse\linalg\dsolve\linsolve.py:318: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)


####################################
Timestep 0 calculation begins


D:\Resource\Reactor Physics\Transient\SCM_test_5_25\main.py:1728: RuntimeWarning: invalid value encountered in true_divide
  disp = abs((dist-dist_last)/dist)


Iteration 1: Eigenvalue k = 0.9751982259747374
Iteration 2: Eigenvalue k = 0.9872255359268153
Iteration 3: Eigenvalue k = 0.9934053092207373
Iteration 4: Eigenvalue k = 0.9970529272938129
Iteration 5: Eigenvalue k = 0.9994367255600837
Iteration 6: Eigenvalue k = 1.0011156504202798
Iteration 7: Eigenvalue k = 1.002366925153452
Iteration 8: Eigenvalue k = 1.0033407706081805
Iteration 9: Eigenvalue k = 1.0041244825553617
Iteration 10: Eigenvalue k = 1.0047717986714324
Iteration 11: Eigenvalue k = 1.0053174486393288
Iteration 12: Eigenvalue k = 1.0057848333251231
Iteration 13: Eigenvalue k = 1.0061902996014023
Iteration 14: Eigenvalue k = 1.0065456323284234
Iteration 15: Eigenvalue k = 1.006859566883381
Iteration 16: Eigenvalue k = 1.0071387403893335
Iteration 17: Eigenvalue k = 1.0073883089290627
Iteration 18: Eigenvalue k = 1.0076123591780561
Iteration 19: Eigenvalue k = 1.007814189631044
Iteration 20: Eigenvalue k = 1.0079965068545154
Iteration 21: Eigenvalue k = 1.0081615650349625
Iter

Iteration 99: Eigenvalue k = 1.009781356097536
Iteration 100: Eigenvalue k = 1.009785139941555
Reached the maximum iteration number100
Predicted reactivity (1st): 0.0038567690082232614
Optimized time step: 0.005629126147932338
Choosen time step: 0.005629126147932338
Time=0.005629126147932338 (s)
Predicted reactivity: 0.0004342047854144992
PKE omega: 2.842170943040401e-14, Real omega: 0
Predicted amplitude frequency: 8.63268669658884
####################################
Timestep 1 calculation begins
Total Q (last): 1.0000000000000002
####################################
Outer iteration 1 begins
Iteration 1: Eigenvalue k = 1.0000801982806482
Iteration 2: Eigenvalue k = 1.000086041956478
Iteration 3: Eigenvalue k = 1.000091991881258
Iteration 4: Eigenvalue k = 1.000097231164207
Iteration 5: Eigenvalue k = 1.0001015696859286
Iteration 6: Eigenvalue k = 1.0001050703658916
Iteration 7: Eigenvalue k = 1.0001078725302948
Iteration 8: Eigenvalue k = 1.000110121069914
Iteration 9: Eigenvalue k =

...

Total Q: 8645.867689822051
####################################
Outer iteration 2 begins
Iteration 1: Eigenvalue k = 0.9999389919219422
Iteration 2: Met the criteria and k = 0.9999383317134514
Outer Iteration 2： kD=0.9999383317134514
Amplitude frequency last:-4.848570889408897
Amplitude frequency:-4.897056598302986
Amplitude frequency new (secant):-4.800085180514808
Total Q: 8662.241095220272
####################################
Outer iteration 3 begins
Iteration 1: Eigenvalue k = 0.9999294061192958
Iteration 2: Met the criteria and k = 0.9999289757061929
Outer Iteration 3： kD=0.9999289757061929
Amplitude frequency last:-4.897056598302986
Amplitude frequency:-4.800085180514808
Amplitude frequency new (secant):-5.536224676519645
Total Q: 8538.71717138402
####################################
Outer iteration 4 begins
Iteration 1: Eigenvalue k = 0.9999925714420868
Iteration 2: Met the criteria and k = 0.9999922161542872
Outer Iteration 4： kD=0.9999922161542872
Amplitude frequency last:-4.8

Total Q: 5780.845656578741
####################################
Outer iteration 2 begins
Iteration 1: Eigenvalue k = 0.9999802702759683
Iteration 2: Met the criteria and k = 0.9999796487211272
Outer Iteration 2： kD=0.9999796487211272
Amplitude frequency last:-3.7360798123244123
Amplitude frequency:-3.7734406104476563
Amplitude frequency new (secant):-3.698719014201168
Total Q: 5787.017237256362
####################################
Outer iteration 3 begins
Iteration 1: Eigenvalue k = 0.9999724085187668
Iteration 2: Met the criteria and k = 0.9999720145839763
Outer Iteration 3： kD=0.9999720145839763
Amplitude frequency last:-3.7734406104476563
Amplitude frequency:-3.698719014201168
Amplitude frequency new (secant):-3.972635360683022
Total Q: 5764.425438943923
####################################
Outer iteration 4 begins
Iteration 1: Eigenvalue k = 0.999996452440871
Iteration 2: Met the criteria and k = 0.9999961588646809
Outer Iteration 4： kD=0.9999961588646809
Amplitude frequency last:-

Iteration 12: Met the criteria and k = 0.999349911543916
Outer Iteration 1： kD=0.999349911543916
Amplitude frequency last:-3.2735624854889402
Amplitude frequency:-2.9975676067858696
Amplitude frequency new (secant):-3.0275432828537285
Total Q: 3758.039100470998
####################################
Outer iteration 2 begins
Iteration 1: Eigenvalue k = 1.0000178730190612
Iteration 2: Met the criteria and k = 1.0000172516192516
Outer Iteration 2： kD=1.0000172516192516
Amplitude frequency last:-2.9975676067858696
Amplitude frequency:-3.0275432828537285
Amplitude frequency new (secant):-2.9675919307180108
Total Q: 3763.6613207870046
####################################
Outer iteration 3 begins
Iteration 1: Eigenvalue k = 1.0000065123853161
Iteration 2: Met the criteria and k = 1.0000061049356133
Outer Iteration 3： kD=1.0000061049356133
Amplitude frequency last:-3.0275432828537285
Amplitude frequency:-2.9675919307180108
Amplitude frequency new (secant):-2.9347571292118575
Total Q: 3766.744122

Iteration 1: Eigenvalue k = 1.0000037867296048
Iteration 2: Met the criteria and k = 1.0000041581465282
Outer Iteration 2： kD=1.0000041581465282
Amplitude frequency last:-1.3496711381729576
Amplitude frequency:-1.3631678495546873
Amplitude frequency new (secant):-1.336174426791228
Total Q: 3209.5057539978097
####################################
Outer iteration 3 begins
Iteration 1: Eigenvalue k = 1.000001036463824
Iteration 2: Met the criteria and k = 1.0000017800787666
Outer Iteration 3： kD=1.0000017800787666
Amplitude frequency last:-1.3631678495546873
Amplitude frequency:-1.336174426791228
Amplitude frequency new (secant):-1.3159687709746428
Total Q: 3210.6178962313866
####################################
Outer iteration 4 begins
Iteration 1: Eigenvalue k = 0.9999997350138826
Iteration 2: Met the criteria and k = 1.000000480430116
Outer Iteration 4： kD=1.000000480430116
Amplitude frequency last:-1.336174426791228
Amplitude frequency:-1.3159687709746428
Amplitude frequency new (secan

Iteration 1: Eigenvalue k = 1.0000222176347746
Iteration 2: Met the criteria and k = 1.0000222581844556
Outer Iteration 2： kD=1.0000222581844556
Amplitude frequency last:-1.4568725369522735
Amplitude frequency:-1.4714412623217963
Amplitude frequency new (secant):-1.442303811582751
Total Q: 2869.39485087949
####################################
Outer iteration 3 begins
Iteration 1: Eigenvalue k = 1.0000196857192354
Iteration 2: Met the criteria and k = 1.0000200854340342
Outer Iteration 3： kD=1.0000200854340342
Amplitude frequency last:-1.4714412623217963
Amplitude frequency:-1.442303811582751
Amplitude frequency new (secant):-1.1729501209835798
Total Q: 2876.762819426786
####################################
Outer iteration 4 begins
Iteration 1: Eigenvalue k = 0.9999941311065743
Iteration 2: Met the criteria and k = 0.9999945940584961
Outer Iteration 4： kD=0.9999945940584961
Amplitude frequency last:-1.442303811582751
Amplitude frequency:-1.1729501209835798
Amplitude frequency new (secan